<span style="color:lightgreen; font-size:30px">**PG103 - Visualización de datos en Geología**</span>
***
<span style="color:gold; font-size:30px">**Visualización de un modelo de bloques en 3D**</span>
***

<span style="font-size:20px"> **Autor: Kevin Alexander Gómez** </span>

<span style="font-size:16px"> **Contacto: kevinalexandr19@gmail.com | [Linkedin](https://www.linkedin.com/in/kevin-alexander-g%C3%B3mez-2b0263111/) | [Github](https://github.com/kevinalexandr19)** </span>
***

Bienvenido al curso PG103 - Visualización de datos en Geología!!!

Vamos a revisar ejemplos de <span style="color:gold">visualización de datos</span> en Geología usando código en Python.\
Es necesario que tengas un conocimiento previo en programación con Python, estadística y geología general.

<span style="color:lightgreen"> Este notebook es parte del proyecto [**Python para Geólogos**](https://github.com/kevinalexandr19/manual-python-geologia), y ha sido creado con la finalidad de facilitar el aprendizaje en Python para estudiantes y profesionales en el campo de la Geología. </span>
***

Antes de empezar tu camino en programación geológica...\
Recuerda que puedes ejecutar un bloque de código usando `Shift` + `Enter`:

In [ ]:
2 + 2

Si por error haces doble clic sobre un bloque de texto (como el que estás leyendo ahora mismo), puedes arreglarlo usando también `Shift` + `Enter`.
***

<a id="parte-1"></a>

### <span style="color:lightgreen">**Modelo de bloques en 3D**</span>
***
En geología, un <span style="color:gold">modelo de bloques</span> es una representación tridimensional de un yacimiento mineral en la que se divide el área en bloques cúbicos o rectangulares.

Cada bloque representa un volumen homogéneo dentro del yacimiento y tiene asignado un valor o conjunto de valores que indican la presencia y cantidad de minerales en ese bloque.

Los modelos de bloques se utilizan en la industria minera para estimar las reservas minerales y planificar la extracción del mineral.\
El proceso de creación de un modelo de bloques comienza con la recopilación de datos geológicos y de exploración del yacimiento, como datos de perforación, análisis geoquímicos y muestreo de superficie.

A partir de estos datos, se utiliza un software especializado para crear un modelo tridimensional del yacimiento dividido en bloques.\
A cada bloque se asigna un valor o conjunto de valores que indican las diferentes variables de interés (ley, porosidad, densidad, etc.).\
Estos valores se pueden determinar a través de diferentes métodos, como la estimación por kriging, la simulación estocástica, entre otros.

Una vez que se ha creado el modelo de bloques, se realizar la estimación de reservas minerales y se planifica la extracción del mineral.

<span style="color:#43c6ac">La precisión y confiabilidad del modelo de bloques depende en gran medida de la calidad de los datos geológicos y de exploración utilizados en su creación, así como de los métodos y supuestos utilizados para asignar valores a los bloques.</span>
***

Empezaremos importando `pandas` para cargar el archivo `blocks.csv` ubicado en la carpeta `files`:
> Nota: debido al límite de tamaño de archivos en GitHub, el archivo `blocks.csv` solo contiene los bloques superficiales del modelo.

In [ ]:
import pandas as pd

modelo = pd.read_csv("files/blocks.csv")

Observamos la información contenida en el modelo de bloques:

In [ ]:
modelo

El modelo contiene 142,960 bloques con las siguientes columnas:
- `X`, `Y`, `Z`: las coordenadas del centroide de cada bloque.
- `LITO`: la litología asociada a cada bloque.
- `Color_LITO`: el color asociado a cada litología.

Usaremos el método `unique` para observar las diferentes litologías presentes en el modelo:

In [ ]:
print(modelo["LITO"].unique())

Y los colores asociados a cada litología:

In [ ]:
print(modelo.groupby("LITO")["Color_LITO"].unique())

Vamos a visualizar el modelo de bloques en 3D usando `PyVista`:

In [ ]:
import pyvista as pv

Primero, crearemos un diccionario asociando los nombres de cada litología con su color respectivo:

In [ ]:
color_lito = modelo.groupby("LITO")["Color_LITO"].unique().to_dict()
color_lito = {k: v[0] for k, v in color_lito.items()}
print(color_lito)

Ahora, visualizaremos el modelo de bloques en 3D usando la función `Plotter` y `PolyData`:

In [ ]:
# Inicializar el plotter
plotter = pv.Plotter()

# Establecer el color de fondo de la visualización
plotter.set_background("black")

# Iterar por litología
for lito, color in color_lito.items():
    # Filtrar puntos por litología
    df = modelo[modelo["LITO"] == lito]

    # Crear PolyData
    puntos = pv.PolyData(df[["X", "Y", "Z"]].values)

    # Crear un cubo que servirá como el glyph para cada punto
    cubo = pv.Cube(x_length=10.0, y_length=10.0, z_length=10.0)

    # Aplicar glyphs para visualizar cada punto como un cubo
    glyphs = puntos.glyph(orient=False, scale=1, geom=cubo)

    # Añadir los glyphs al plotter con el color correspondiente
    plotter.add_mesh(glyphs, color=color, label=lito)

# Personalizar la leyenda
legend_params = {
    "labels": None,         # Usar las etiquetas especificadas en add_mesh
    "bcolor": "black",      # Color de fondo blanco
    "border": True,         # Habilitar borde
    "size": (0.2, 0.2),     # Tamaño de la leyenda
    "loc": "upper right",   # Ubicación de la leyenda
    "face": "rectangle",    # Forma de los íconos de la leyenda
}

# Añadir leyenda para identificar cada litología por color
plotter.add_legend(**legend_params)

# Mostrar la cuadrícula (grid) con líneas de color blanco
plotter.show_grid(color="white", font_size=10)

# Esto coloca la cámara a cierta distancia, mirando hacia una posición específica, con (0, 0, 1) como el vector de vista vertical
camera_distance = (4800, -3200, 4500) # x, y, z
camera_view = (0, 1600, 0) # x, y, z
camera_position = [camera_distance, camera_view, (0, 0, 1)]

# Mostrar la visualización
plotter.camera_position = camera_position
plotter.show()

***